In [1]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker
import torch
import scib_metrics
%matplotlib inline
import sys
sys.path.append('..')
from functions import *

/local/data/scvi-tools/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/local/data/scvi-tools/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  dispatcher = jit(_target='npyufunc',
/local/data/anaconda3/envs/BT/lib/python3.11/site-pac

In [2]:
adata = anndata.read_h5ad(filename="../data/adataLung4.h5ad")
adata

AnnData object with n_obs × n_vars = 32472 × 4000
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'pca'
    obsm: 'Scanorama', 'Unintegrated', 'X_pca'
    varm: 'PCs'
    layers: 'counts'

In [3]:
sdnormalAdata, vaeSD = trainModelBenchmark(adata.copy(), "sdnormal", log=True,logname="sdLungBench",save="models/LBM_StandardNormal")

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/sdLungBench
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [12:59<00:00,  1.94s/it, v_num=0, train_loss_step=787, train_loss_epoch=905]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [12:59<00:00,  1.95s/it, v_num=0, train_loss_step=787, train_loss_epoch=905]


In [4]:
mogAdata, vaeMG = trainModelBenchmark(adata.copy(), "mixofgaus",log=True,logname="mogLungBench",save="models/LBM_MixtureOfGaussians")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/mogLungBench
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/400:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch 400/400: 100%|██████████| 400/400 [14:59<00:00,  2.30s/it, v_num=0, train_loss_step=976, train_loss_epoch=899]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [14:59<00:00,  2.25s/it, v_num=0, train_loss_step=976, train_loss_epoch=899]


In [5]:
vampAdata, vaeVP = trainModelBenchmark(adata.copy(), "vamp",log=True,logname="vampLungBench",save="models/LBM_Vamp")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/vampLungBench
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/400:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch 400/400: 100%|██████████| 400/400 [16:31<00:00,  2.52s/it, v_num=0, train_loss_step=873, train_loss_epoch=900]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [16:31<00:00,  2.48s/it, v_num=0, train_loss_step=873, train_loss_epoch=900]


In [6]:
flowAdata, vaeNF = trainModelBenchmark(adata.copy(), 'normalflow', log=True, logname="flowLungBench",save="models/LBM_NormalFlow")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/flowLungBench
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/400:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch 400/400: 100%|██████████| 400/400 [23:49<00:00,  3.59s/it, v_num=0, train_loss_step=881, train_loss_epoch=902]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [23:49<00:00,  3.57s/it, v_num=0, train_loss_step=881, train_loss_epoch=902]


In [3]:
vaeSD = scvi.model.SCVI.load("models/LBM_StandardNormal")
vaeMG = scvi.model.SCVI.load("models/LBM_MixtureOfGaussians")
vaeVP = scvi.model.SCVI.load("models/LBM_Vamp")
vaeNF = scvi.model.SCVI.load("models/LBM_NormalFlow")

INFO     File models/LBM_StandardNormal/model.pt already downloaded                                                


INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.


INFO     File models/LBM_MixtureOfGaussians/model.pt already downloaded                                            
INFO     File models/LBM_Vamp/model.pt already downloaded                                                          
INFO     File models/LBM_NormalFlow/model.pt already downloaded                                                    


In [4]:
allAdata = adata.copy()
allAdata.obsm["SN"] = vaeSD.get_latent_representation()
allAdata.obsm["MG"] = vaeMG.get_latent_representation()
allAdata.obsm["VP"] = vaeVP.get_latent_representation()
allAdata.obsm["NF"] = vaeNF.get_latent_representation()

In [5]:
plotBenchmarkResults(allAdata, keys=["Unintegrated","Scanorama", "SN", "MG", "VP", "NF"], save_dir="Benchmarks/")

Embeddings:   0%|          | 0/6 [00:00<?, ?it/s]INFO:root:isolated labels: no more than 4 batches per label
INFO:root:Type 1: 0.4629906415939331
Embeddings:  17%|█▋        | 1/6 [00:47<03:56, 47.20s/it]INFO:root:isolated labels: no more than 4 batches per label
INFO:root:Type 1: 0.5239854902029037
Embeddings:  33%|███▎      | 2/6 [04:12<09:19, 139.97s/it]INFO:root:isolated labels: no more than 4 batches per label
INFO:root:Type 1: 0.5552385747432709
Embeddings:  50%|█████     | 3/6 [04:51<04:42, 94.07s/it] INFO:root:isolated labels: no more than 4 batches per label
INFO:root:Type 1: 0.4943151930347085
Embeddings:  67%|██████▋   | 4/6 [05:27<02:22, 71.06s/it]INFO:root:isolated labels: no more than 4 batches per label
INFO:root:Type 1: 0.45720408111810684
Embeddings:  83%|████████▎ | 5/6 [06:02<00:58, 58.12s/it]INFO:root:isolated labels: no more than 4 batches per label
INFO:root:Type 1: 0.4712014328688383
Embeddings: 100%|██████████| 6/6 [06:44<00:00, 67.43s/it]


In [1]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Launching TensorBoard...

In [ ]:
%reload_ext tensorboard